chunk_size = 10000
for chunk in pd.read_json('Call-Rails-2025-01-23.json', lines =True, chunksize=chunk_size):
    data = [tuple(row) for row in chunk.to_numpy()]

In [ ]:
record = data[0]

In [ ]:
record_list = list(record)

In [ ]:
# Move 'pk' to 'fields'
for item in record_list:
    item['fields']['pk'] = item['pk']
    del item['pk']

In [ ]:
import pyodbc
from datetime import datetime
def drop_create_insert_sql_tables(conn_str, schema_name, data_list_name):
    conn = pyodbc.connect(conn_str)
    cursor= conn.cursor()

    # Get the fields from the first tracker
    fields = data_list_name[0]['fields']
    field_names = fields.keys()
    date_stamp = datetime.now().strftime("%Y%m%d")
    table_name = f"{data_list_name[0]['model'].split('.')[-1]}_{date_stamp}"
    


    # Drop table if exists
    truncate_query = f"Drop Table if exists {schema_name}.{table_name}"
    cursor.execute(truncate_query)
    conn.commit()

    # Create table dynamically
    create_table_query = f"CREATE TABLE {schema_name}.{table_name} ("
    for field in field_names:
        if isinstance(fields[field], int):
            create_table_query += f"{field} INT, "
        elif isinstance(fields[field], bool):
            create_table_query += f"{field} BIT, "
        else:
            create_table_query += f"{field} VARCHAR(MAX), "
        
    create_table_query = create_table_query.rstrip(", ") + ")"
    cursor.execute(create_table_query)
    conn.commit()



    # Insert data dynamically
    insert_query = f"INSERT INTO {schema_name}.{table_name} ({', '.join(field_names)}) VALUES ({', '.join(['?' for _ in field_names])})"

    for tracker in data_list_name:
        fields = tracker['fields']
        values = tuple(fields[field] for field in field_names)
        cursor.execute(insert_query, values)
    conn.commit()
    

In [ ]:
# Define table names as a set
table_names_wo_schema = set()

for item in record_list:
    table_names_wo_schema.add(item['model'].split('.')[-1])

In [ ]:
# Initialize a dictionary to store lists for each table
tables = {table_name_wo_schema: [] for table_name_wo_schema in table_names_wo_schema}

# Iterate through the record list
for item in record_list:
    table_name_wo_schema = item['model'].split('.')[-1]
    if table_name_wo_schema in tables:
        tables[table_name_wo_schema].append(item)
        
# Access the lists for each table dynamically
for table_name_wo_schema in table_names_wo_schema:
    globals()[table_name_wo_schema] = tables[table_name_wo_schema]
# Close the connection
cursor.close()
conn.close()

In [ ]:
server = 'DB06.petvetcarecenters.com' 
database = 'ReportingPV' 
username = 'PetVet_RW' 
password = 'p3tv3tc@r3c3nt3rs'  
con_str = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
schema_name = "callrails"
for table_name_wo_schema in table_names_wo_schema:
    drop_create_insert_sql_tables(con_str, schema_name, globals()[table_name_wo_schema])

source = []
for item in record_list:
    if item['model'].split('.')[-1] == "source":
        source.append(item)

callrecord = []
for item in record_list:
    if item['model'].split('.')[-1]=='callrecord':
        callrecord.append(item)


server = 'DB06.petvetcarecenters.com' 
database = 'ReportingPV' 
username = 'PetVet_RW' 
password = 'p3tv3tc@r3c3nt3rs'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


# Get the fields from the first tracker
fields = trackers[0]['fields']
field_names = fields.keys()
table_name = trackers[0]['model'].split('.')[-1]


# Drop table if exists
table_name = trackers[0]['model'].split('.')[-1]
truncate_query = f"Drop Table if exists callrails.{table_name}"
cursor.execute(truncate_query)
cnxn.commit()

# Create table dynamically
create_table_query = f"CREATE TABLE callrails.{table_name} ("
for field in field_names:
    if isinstance(fields[field], int):
        create_table_query += f"{field} INT, "
    elif isinstance(fields[field], bool):
        create_table_query += f"{field} BIT, "
    else:
        create_table_query += f"{field} VARCHAR(MAX), "
create_table_query = create_table_query.rstrip(", ") + ")"
print(create_table_query)
cursor.execute(create_table_query)
cnxn.commit()

# Insert data dynamically
insert_query = f"INSERT INTO callrails.{table_name} ({', '.join(field_names)}) VALUES ({', '.join(['?' for _ in field_names])})"

for tracker in trackers:
    fields = tracker['fields']
    values = tuple(fields[field] for field in field_names)
    cursor.execute(insert_query, values)
cnxn.commit()
# print(insert_query)


In [2]:
# cursor.executemany(insert_query, data_to_insert_test)
# cnxn.commit()  # Commit the transaction
# Close the connection
cursor.close()
conn.close()

NameError: name 'cursor' is not defined